In [111]:
def new_machine():
    return {'pc': 0, 
            'a': 0,
            'b': 0, 
            'c': 0,
            'instructions': []}

In [193]:
def show_machine(machine):
    return ', '.join('{}: {}'.format(sk, machine[int(sk) if sk.isnumeric() else sk]) 
                     for sk in sorted(str(k) for k in machine)
                     if sk != 'instructions')

In [112]:
def inc(reg, machine):
    machine[reg] += 1
    machine['pc'] += 1

In [113]:
def dec(reg, machine):
    machine[reg] -= 1
    machine['pc'] += 1

In [114]:
def jmp(addr, machine):
    machine['pc'] += addr

In [115]:
def jpz(reg, addr, machine):
    if machine[reg] == 0:
        machine['pc'] += addr
    else:
        machine['pc'] += 1

In [116]:
def set_literal(reg, literal, machine):
    machine[reg] = literal
    machine['pc'] += 1

In [117]:
def cpy(from_reg, to_reg, machine):
    machine[to_reg] = machine[from_reg]
    machine['pc'] += 1

In [129]:
def sto(reg, addr, machine):
    machine[addr] = machine[reg]
    machine['pc'] += 1

In [130]:
def ld(reg, addr, machine):
    if addr in machine:
        machine[reg] = machine[addr]
    else:
        machine[reg] = 0
    machine['pc'] += 1

In [131]:
instruction_table = {'inc': inc, 'dec': dec, 'jmp': jmp,
                    'jpz': jpz, 'set': set_literal, 'cpy': cpy,
                    'sto': sto, 'ld': ld}
numeric_args_table = {'jmp': [0], 'jpz': [1], 'set': [1], 'sto': [1], 'ld': [1]}

In [121]:
def parse(instruction):
    words = instruction.split()
    instr = words[0]
    args = words[1:]
    if instr in numeric_args_table:
        for p in numeric_args_table[instr]:
            args[p] = int(args[p])
    return instruction_table[instr], args

In [122]:
m = new_machine()
inc('a', m)
cargs = ['a', 'b']
cpy(*cargs, m)
inc('a', m)
m

{'a': 2, 'b': 1, 'c': 0, 'instructions': [], 'pc': 3}

In [123]:
def program_from_instructions(prog, machine):
    machine['instructions'] = [parse(instr) for instr in prog]

In [124]:
def program_from_listing(listing, machine):
    program_from_instructions([i for i in listing.split('\n') if i], machine)

In [156]:
def run(machine, initial_state=None, trace=False):
    if initial_state:
        machine.update(initial_state)
    while machine['pc'] < len(machine['instructions']):
        if trace:
            print(show_machine(machine))
        cmd, args = machine['instructions'][machine['pc']]
        cmd(*args, machine)

In [146]:
def execute(listing, initial_state=None, trace=False):
    m = new_machine()
    program_from_listing(listing, m)
    run(m, initial_state=initial_state, trace=trace)
    return m

In [127]:
program = """
inc a
inc a
cpy a b
inc a
"""
execute(program)

{'a': 3,
 'b': 2,
 'c': 0,
 'instructions': [(<function __main__.inc>, ['a']),
  (<function __main__.inc>, ['a']),
  (<function __main__.cpy>, ['a', 'b']),
  (<function __main__.inc>, ['a'])],
 'pc': 4}

In [143]:
program = """
set a 10
dec a
inc b
sto b 1
jpz a 2
jmp -4
"""
# m = new_machine()
# program_from_listing(program, m)
# run(m)
execute(program, initial_state={'c': 20})

{'c': 20,
 'instructions': [(<function __main__.set_literal>, ['a', 10]),
  (<function __main__.dec>, ['a']),
  (<function __main__.inc>, ['b']),
  (<function __main__.sto>, ['b', 1]),
  (<function __main__.jpz>, ['a', 2]),
  (<function __main__.jmp>, [-4])],
 1: 10,
 'b': 10,
 'pc': 6,
 'a': 0}

In [159]:
program = """
cpy c a
set b 0
dec a
jpz b 3
dec b
jmp 2
inc b
jpz a 3
jmp -6
"""
# m = new_machine()
# program_from_listing(program, m)
# run(m)
execute(program, initial_state={'c': 5})

{'a': 0,
 'b': 1,
 'c': 5,
 'instructions': [(<function __main__.cpy>, ['c', 'a']),
  (<function __main__.set_literal>, ['b', 0]),
  (<function __main__.dec>, ['a']),
  (<function __main__.jpz>, ['b', 3]),
  (<function __main__.dec>, ['b']),
  (<function __main__.jmp>, [2]),
  (<function __main__.inc>, ['b']),
  (<function __main__.jpz>, ['a', 3]),
  (<function __main__.jmp>, [-6])],
 'pc': 10}

In [196]:
program = """
sto a 10
set c 0
set b 0
dec a
jpz b 4
dec b
inc c
jmp 2
inc b
jpz a 2
jmp -7
"""
# m = new_machine()
# program_from_listing(program, m)
# run(m)
show_machine(execute(program, initial_state={'a': 10}))

'10: 10, a: 0, b: 0, c: 5, pc: 11'